In [123]:
import os
import time
import random
import pandas as pd
from datetime import date
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
import requests

class AmazonProductScraper:
    """
    AmazonProductScraper is a class designed to scrape product information from Amazon Egypt. 
    It automates the process of navigating product pages, extracting detailed product data, and saving the information in a structured format. 
    The class uses Selenium for browser automation and BeautifulSoup for parsing HTML content.

    Methods:
        - __init__(): Initializes the scraper, setting up necessary variables like the base URL and the web driver.
        - open_browser(): Configures and opens a Chrome browser in headless mode for scraping.
        - get_whole_rating(soup): Extracts the overall product rating from the HTML content of a product page.
        - get_category_url(): Constructs and returns the Amazon URL for a specific product or category.
        - extract_webpage_information(): Extracts product data from the current page source.
        - extract_product_information(page_results): Extracts detailed information for each product on a page.
        - navigate_to_other_pages(category_url): Navigates through all pages of search results for a given category, extracting product data.
        - product_information_spreadsheet(records): Saves the extracted product data to a CSV file, ensuring that incomplete records are removed.
        - get_asin_from_url(url): Extracts the ASIN (Amazon Standard Identification Number) from a product URL.
        - get_unit_price(soup): Extracts the unit price from the HTML content of a product page.
        - get_category(soup): Extracts the category and subcategory of a product from its HTML content.
        - get_product_name(soup): Extracts the product name from the HTML content of a product page.
        - get_promo_details(soup): Extracts promotional details from the HTML content of a product page.
        - get_price(soup): Extracts the price before and after promotion from the HTML content of a product page.
        - get_phone_specifications(soup): Extracts phone-specific details like type, storage, operating system, RAM, and color.
        - get_stock_and_purchases(soup): Extracts stock availability and purchase details from the HTML content of a product page.
        - get_description(soup): Extracts the product description from the HTML content of a product page.
        - get_image_links(soup): Extracts image URLs from the HTML content of a product page.
        - get_reviews(soup): Extracts customer reviews from the HTML content of a product page.

    Usage:
        This class is intended to be run as a standalone script. When executed, it will prompt the user to input a product or category name,
        then it will scrape data for that category and save it to a CSV file. The process involves navigating through all search result pages,
        extracting relevant data, and handling missing information to ensure the output file is clean and complete.
    """
    def __init__(self):
        """
        Initializes the AmazonProductScraper class, setting the driver to None and defining the base URL for Amazon.
        """
        self.driver = None
        self.base_url = "https://www.amazon.eg/s?k="

    def open_browser(self):
        """
        Opens a Chrome browser in headless mode using Selenium. 
        Configures the browser with necessary options and installs the ChromeDriver.
        """
        options = Options()
        options.add_argument("--headless")  # Run in headless mode
        options.add_argument("--disable-infobars")
        options.add_argument("--disable-extensions")
        options.add_argument("--log-level=OFF")
        options.add_experimental_option('excludeSwitches', ['enable-logging'])

        service = Service(ChromeDriverManager().install())
        self.driver = webdriver.Chrome(service=service, options=options)

    def get_whole_rating(self, soup):
        """
        Extracts the overall rating of a product from the given BeautifulSoup object.

        Args:
            soup (BeautifulSoup): The parsed HTML of the product page.

        Returns:
            str: The overall rating of the product. Returns an empty string if not found.
        """
        try:
            rating_element = soup.find("span", attrs={'class': 'a-icon-alt'})
            whole_rating = rating_element.text.strip() if rating_element else ""
        except AttributeError:
            whole_rating = ""
        return whole_rating

    
    
    def get_category_url(self):
        """
        Constructs and returns the URL for the specified product/category search on Amazon Egypt.
        Navigates to the category page using Selenium.

        Returns:
            tuple: The constructed category URL and the category name entered by the user.
        """
        category_name = input("\n>> Enter the product/category to be searched: ")
        formatted_category_name = category_name.replace(" ", "+")
        category_url = f"{self.base_url}{formatted_category_name}&ref=nb_sb_noss"
        print(">> Category URL: ", category_url)
        self.driver.get(category_url)
        time.sleep(3)
        return category_url, category_name

    def extract_webpage_information(self):
        """
        Extracts product information from the current page source using BeautifulSoup.

        Returns:
            list: A list of div elements containing product information.
        """
        soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        page_results = soup.find_all('div', {'data-component-type': 's-search-result'})
        return page_results

    def extract_product_information(self, page_results):
        """
        Extracts detailed information for each product found on the current page.

        Args:
            page_results (list): A list of div elements containing product information.

        Returns:
            list: A list of dictionaries, each containing detailed product information.
        """
        records = []
        for item in page_results:
            link = item.find("a", attrs={'class': 'a-link-normal s-no-outline'})['href']
            product_url = link if link.startswith("https://") else "https://www.amazon.eg" + link

            self.driver.get(product_url)
            product_soup = BeautifulSoup(self.driver.page_source, "html.parser")

            category, subcategory = self.get_category(product_soup)
            product_name = self.get_product_name(product_soup)
            promo_details = self.get_promo_details(product_soup)
            price_before_promo, price_after_promo = self.get_price(product_soup)
            phone_type, operating_system, storage, ram_memory, color = self.get_phone_specifications(product_soup)
            stock, purchases = self.get_stock_and_purchases(product_soup)
            reviews = self.get_reviews(product_soup)
            description = self.get_description(product_soup)
            asin = self.get_asin_from_url(product_url)
            unit_price = self.get_unit_price(product_soup)
            image_links = self.get_image_links(product_soup)
            whole_rating = self.get_whole_rating(product_soup)  
            
            review_dates = " | ".join([review['review_date'] for review in reviews])
            review_ratings = " | ".join([review['review_rating'] for review in reviews])
            reviewer_names = " | ".join([review['reviewer_name'] for review in reviews])
            review_texts = " | ".join([review['review_text'] for review in reviews])

            product_information = {
                "category": category, "subcategory": subcategory, "product_name": product_name,
                "promo_details": promo_details, "price_before_promo": price_before_promo,
                "price_after_promo": price_after_promo, "phone_type": phone_type,
                "operating_system": operating_system, "storage": storage, "ram_memory": ram_memory,
                "color": color, "stock": stock, "purchases": purchases, "description": description,
                "asin": asin, "unit_price": unit_price, "image_links": " | ".join(image_links),
                "product_url": product_url, "review_dates": review_dates, "review_ratings": review_ratings,
                "reviewer_names": reviewer_names, "review_texts": review_texts, "whole_rating": whole_rating}

            records.append(product_information)

            time.sleep(random.uniform(2, 4))

        return records

    def navigate_to_other_pages(self, category_url):
        """
        Navigates through all pages of search results for the given category and extracts product information from each page.

        Args:
            category_url (str): The URL of the category search page.

        Returns:
            list: A list of dictionaries containing product information from all pages.
        """
        all_records = []

        try:
            max_pages_xpath = "//span[@class='s-pagination-item s-pagination-disabled']"
            max_pages = self.driver.find_element(By.XPATH, max_pages_xpath).text
            max_pages = int(max_pages)
        except NoSuchElementException:
            max_pages = 1

        print(f">> Maximum Pages: {max_pages}")

        for page in range(1, max_pages + 1):
            next_page_url = f"{category_url}&page={page}"
            self.driver.get(next_page_url)
            page_results = self.extract_webpage_information()
            records = self.extract_product_information(page_results)
            all_records.extend(records)
            print(f">> Page {page} - webpage information extracted")

        self.driver.quit()
        return all_records
    

    def product_information_spreadsheet(self, records):
        """
        Saves the extracted product information to a CSV file.
        Ensures that no products without a name are included and appends new records to the existing file.

        Args:
            records (list): A list of dictionaries containing product information.
        """
        file_name = "products.csv" 
        df = pd.DataFrame(records)

        threshold = len(df.columns) * 0.5
        df = df.dropna(thresh=threshold)
        
        if 'product_name' in df.columns:
            df = df[df['product_name'].notna()]

        if os.path.exists(file_name):
            df.to_csv(file_name, mode='a', index=False, header=False, encoding='utf-8')
        else:
            df.to_csv(file_name, index=False, encoding='utf-8')
        
        print(f">> Information about the product is stored in {file_name}\n")
        os.startfile(file_name)
        
    def get_asin_from_url(self, url):
        """
        Extracts the ASIN (Amazon Standard Identification Number) from the product page URL.

        Args:
            url (str): The URL of the product page.

        Returns:
            str: The ASIN of the product, or None if not found.
        """
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        try:
            asin_element = soup.find('table', {'id': 'productDetails_detailBullets_sections1'}).find('td', {'class': 'a-size-base prodDetAttrValue'})
            asin = asin_element.text.strip() if asin_element else None
        except AttributeError:
            asin = None
        return asin

    def get_unit_price(self, soup):
        """
        Extracts the unit price of the product from the given BeautifulSoup object.

        Args:
            soup (BeautifulSoup): The parsed HTML of the product page.

        Returns:
            str: The unit price of the product. Returns an empty string if not found.
        """
        try:
            price_symbol = soup.find('span', {'class': 'a-price-symbol'}).text.strip()
            price_whole = soup.find('span', {'class': 'a-price-whole'}).text.strip()
            price_fraction = soup.find('span', {'class': 'a-price-fraction'}).text.strip()
            unit_price = f"{price_symbol}{price_whole}.{price_fraction}"
        except AttributeError:
            unit_price = ""
        return unit_price

    def get_category(self, soup):
        """
        Extracts the category and subcategory of the product from the given BeautifulSoup object.

        Args:
            soup (BeautifulSoup): The parsed HTML of the product page.

        Returns:
            tuple: The category and subcategory of the product.
        """
        try:
            breadcrumb = soup.find('div', attrs={'id': 'wayfinding-breadcrumbs_feature_div'})
            categories = breadcrumb.find_all('li')
            category = categories[0].text.strip() if categories else ""
            subcategory = categories[-1].text.strip() if categories else "mobile phones"
        except AttributeError:
            category = ""
            subcategory = "mobile phones"
        return category, subcategory

    def get_product_name(self, soup):
        """
        Extracts the product name from the Amazon product page.

        Parameters:
        - soup (BeautifulSoup): Parsed HTML of the product page.

        Returns:
        - str: The name of the product, or an empty string if not found.
        """
        try:
            product_name = soup.find("span", attrs={"id": 'productTitle'}).text.strip()
        except AttributeError:
            product_name = ""
        return product_name

    def get_promo_details(self, soup):
        """
        Extracts the product name from the Amazon product page.

        Parameters:
        - soup (BeautifulSoup): Parsed HTML of the product page.

        Returns:
        - str: The name of the product, or an empty string if not found.
        """
        try:
            promo = soup.find('span', class_='a-size-large a-color-price savingPriceOverride aok-align-center reinventPriceSavingsPercentageMargin savingsPercentage').text.strip()
        except AttributeError:
            promo = ""
        return promo

    def get_price(self, soup):
        """
        Extracts the price before and after promotion from the Amazon product page.

        Parameters:
        - soup (BeautifulSoup): Parsed HTML of the product page.

        Returns:
        - tuple: A tuple containing the price before promotion and the price after promotion as strings.
        """
        try:
            price_before = soup.find("span", attrs={'class': 'a-price a-text-price'}).find("span", attrs={'class': 'a-offscreen'}).text.strip()
        except AttributeError:
            price_before = ""

        try:
            price_symbol = soup.find("span", attrs={'class': 'a-price-symbol'}).text.strip()
            price_whole = soup.find("span", attrs={'class': 'a-price-whole'}).text.strip()
            price_fraction = soup.find("span", attrs={'class': 'a-price-fraction'}).text.strip()
            price_after = f"{price_symbol}{price_whole}.{price_fraction}"
        except AttributeError:
            price_after = ""
        
        return price_before, price_after

    def get_phone_specifications(self, soup):
        """
        Extracts phone-specific details like phone type, storage, operating system, RAM memory, and color.

        Parameters:
        - soup (BeautifulSoup): Parsed HTML of the product page.

        Returns:
        - tuple: A tuple containing phone type, operating system, storage, RAM memory, and color as strings.
        """
        try:
            phone_type = soup.find("tr", attrs={'class': 'a-spacing-small po-brand'}).find("span", attrs={'class': 'a-size-base po-break-word'}).text.strip()
        except AttributeError:
            phone_type = ""

        try:
            storage = soup.find("tr", attrs={'class': 'a-spacing-small po-memory_storage_capacity'}).find("span", attrs={'class': 'a-size-base po-break-word'}).text.strip()
        except AttributeError:
            storage = ""

        try:
            operating_system = soup.find("tr", attrs={'class': 'a-spacing-small po-operating_system'}).find("span", attrs={'class': 'a-size-base po-break-word'}).text.strip()
        except AttributeError:
            operating_system = ""

        try:
            ram_memory = soup.find("tr", attrs={'class': 'a-spacing-small po-ram_memory.installed_size'}).find("span", attrs={'class': 'a-size-base po-break-word'}).text.strip()
        except AttributeError:
            ram_memory = ""

        try:
            color = soup.find("tr", attrs={'class': 'a-spacing-small po-color'}).find("span", attrs={'class': 'a-size-base po-break-word'}).text.strip()
        except AttributeError:
            color = ""
        
        return phone_type, operating_system, storage, ram_memory, color

    def get_stock_and_purchases(self, soup):
        """
        Extracts stock availability and purchase details from the Amazon product page.

        Parameters:
        - soup (BeautifulSoup): Parsed HTML of the product page.

        Returns:
        - tuple: A tuple containing stock availability and purchase details as strings.
        """
        try:
            stock = soup.find("div", attrs={'id': 'availability'}).find("span").text.strip()
        except AttributeError:
            stock = ""

        try:
            purchases = soup.find("span", attrs={'id': 'social-proofing-faceout-title-tk_bought'}).text.strip()
        except AttributeError:
            purchases = ""
        
        return stock, purchases

    def get_description(self, soup):
        """
        Extracts the product's description from the Amazon product page.

        Parameters:
        - soup (BeautifulSoup): Parsed HTML of the product page.

        Returns:
        - str: Product description as a string, or an empty string if not found.
        """
        try:
            description = soup.find("div", attrs={'id': 'feature-bullets'}).find("span", attrs={'class': 'a-list-item'}).text.strip()
        except AttributeError:
            description = ""
        return description

    def get_image_links(self, soup):
        """
        Extracts image links of the product from the Amazon product page.

        Parameters:
        - soup (BeautifulSoup): Parsed HTML of the product page.

        Returns:
        - list: A list of image URLs as strings.
        """
        image_links = []
        try:
            image_data = soup.find('div', attrs={'id': 'altImages'}).find_all('img')
            for img in image_data:
                image_links.append(img['src'])
        except AttributeError:
            pass
        return image_links


    def get_reviews(self, soup):
        """
        Extracts customer reviews from the Amazon product page.

        Parameters:
        - soup (BeautifulSoup): Parsed HTML of the product page.

        Returns:
        - list: A list of dictionaries, each containing review details such as review date, rating, reviewer name, and review text.
        """
        reviews = []
        try:
            review_divs = soup.find_all("div", attrs={'data-hook': 'review'})

            for div in review_divs:
                review = {
                    "review_date": div.find("span", attrs={'data-hook': 'review-date'}).text.strip(),
                    "review_rating": div.find("i", attrs={'data-hook': 'review-star-rating'}).text.strip(),
                    "reviewer_name": div.find("span", attrs={'class': 'a-profile-name'}).text.strip(),
                    "review_text": div.find("span", attrs={'data-hook': 'review-body'}).text.strip()
                }
                reviews.append(review)
        except AttributeError:
            pass
        return reviews

if __name__ == '__main__': 
    scraper = AmazonProductScraper()
    scraper.open_browser()
    category_url, category_name = scraper.get_category_url()
    records = scraper.navigate_to_other_pages(category_url)
    scraper.product_information_spreadsheet(records)  



>> Enter the product/category to be searched: iphone 15
>> Category URL:  https://www.amazon.eg/s?k=iphone+15&ref=nb_sb_noss
>> Maximum Pages: 7
>> Page 1 - webpage information extracted
>> Page 2 - webpage information extracted
>> Page 3 - webpage information extracted
>> Page 4 - webpage information extracted
>> Page 5 - webpage information extracted
>> Page 6 - webpage information extracted
>> Page 7 - webpage information extracted
>> Information about the product is stored in products.csv



In [124]:
#cleaning data
import pandas as pd

def clean_csv_file(file_path):
    # Read the CSV file
    df = pd.read_csv(file_path)

    # Function to keep only 'جنيه' in the 'unit_price' column
    def clean_unit_price(value):
        if isinstance(value, str) and 'جنيه' in value:
            return 'جنيه'
        return value

    def clean_price_before_promo(value):
        if isinstance(value, str):
            return value.replace('جنيه', '').replace(',', '').strip()
        return value

    def clean_price_after_promo(value):
        if isinstance(value, str):
            return value.replace('جنيه', '').replace(',', '').strip()
        return value

    if 'unit_price' in df.columns:
        df['unit_price'] = df['unit_price'].apply(clean_unit_price)

    if 'price_before_promo' in df.columns:
        df['price_before_promo'] = df['price_before_promo'].apply(clean_price_before_promo)

    if 'price_after_promo' in df.columns:
        df['price_after_promo'] = df['price_after_promo'].apply(clean_price_after_promo)

    df = df.dropna(subset=['product_name'])

    try:
        df.to_csv(file_path, index=False, encoding='utf-8')
        print(f">> Cleaned and saved: {file_path}")
    except PermissionError:
        new_file_path = file_path.replace('.csv', '_cleaned.csv')
        df.to_csv(new_file_path, index=False, encoding='utf-8')
        print(f">> Permission denied. Saved cleaned data to: {new_file_path}")

if __name__ == "__main__":
    file_path = 'products.csv'
    clean_csv_file(file_path)


>> Cleaned and saved: products.csv


In [125]:
import pandas as pd
df = pd.read_csv('products.csv')
df.columns

Index(['category', 'subcategory', 'product_name', 'promo_details',
       'price_before_promo', 'price_after_promo', 'phone_type',
       'operating_system', 'storage', 'ram_memory', 'color', 'stock',
       'purchases', 'description', 'asin', 'unit_price', 'image_links',
       'product_url', 'review_dates', 'review_ratings', 'reviewer_names',
       'review_texts', 'whole_rating'],
      dtype='object')

In [126]:
df

,category,subcategory,product_name,promo_details,price_before_promo,price_after_promo,phone_type,operating_system,storage,ram_memory,...,description,asin,unit_price,image_links,product_url,review_dates,review_ratings,reviewer_names,review_texts,whole_rating
0,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,موبايل سامسونج جالاكسي A15 ثنائي شرائح الاتصال...,-19‎%‎,‎10270.00‎,8299.‎.00,سامسونج,Android 13.0,256 غيغابايت,8 غيغابايت,...,الشاشة: تتميز شاشة سوبر اموليد مقاس 6.5 بوصة ل...,B0CQJZ76X9,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D9%85%D9%88%D8%A8%D8%A7...,تمت كتابة هذا التقييم في مصر في 19 أغسطس 2024 ...,5.0 من 5 نجوم | 5.0 من 5 نجوم | 1.0 من 5 نجوم ...,محمد حسين | إيناس ل. | Eng Ahmed Mesbah | افشل...,هاتف جيد وجاء كما هو مطلوب شكرا امازون\nاقرأ ا...,3.9 من 5 نجوم
1,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,جوال سامسونج جالكسي ايه 55 الجيل الخامس 5G، ثن...,NaN,NaN,19099.‎.00,سامسونج,Android 11.0,256 غيغابايت,8 غيغابايت,...,العلامة التجارية: سامسونج,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D8%B3%D8%A7%D9%85%D8%B3...,تمت كتابة هذا التقييم في مصر في 14 أغسطس 2024 ...,5.0 من 5 نجوم | 5.0 من 5 نجوم | 1.0 من 5 نجوم ...,Ahmed Baraka | Abanoub | أبانوب ميشيل | Buyer ...,موبايل إمكانياته محترمه ومثالي للاستخدام اليوم...,4.0 من 5 نجوم
2,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,هاتف ذكي ريدمي A3 بذاكرة RAM 3GB وذاكرة تخزين ...,-18‎%‎,‎4887.72‎,3999.‎.00,شاومي,Android 10.0,64 غيغابايت,NaN,...,يدعمي شريحتي اتصال وميكرو اس دي، يدعم نطاقات ش...,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D9%88%D8%A8%D8%B7%D8%A7...,تمت كتابة هذا التقييم في مصر في 24 يوليو 2024 ...,5.0 من 5 نجوم | 3.0 من 5 نجوم | 1.0 من 5 نجوم ...,Ahmed abdallah | Hadeer Haggag | ياسر عبدالحى ...,ممتاز جدا\nاقرأ المزيد | مناسب بالنسبه للسعر ل...,3.5 من 5 نجوم
3,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,موبايل سمارت سامسونج جالاكسي A55 الجيل الخامس ...,-23‎%‎,‎23499.00‎,17995.‎.00,سامسونج,Android 11.0,256 غيغابايت,8 غيغابايت,...,العلامة التجارية: سامسونج,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D8%B3%D8%A7%D9%85%D8%B3...,تمت كتابة هذا التقييم في مصر في 27 يونيو 2024,5.0 من 5 نجوم,أيمن خطاب,منتج جيد وبالتقسيط بدون فوائد ومغلق بشكل جيد\n...,4.1 من 5 نجوم
4,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,جوال شاومي ريدمي 13C ثنائي شرائح الاتصال بذاكر...,-13‎%‎,‎6777.00‎,5866.‎.24,شاومي,ام اي يو اي 14,128 غيغابايت,6 غيغابايت,...,شاشة عرض 6.74 بوصة 90 هرتز,B0CT3MBS1L,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D8%B4%D8%A7%D9%88%D9%85...,تمت كتابة هذا التقييم في مصر في 16 أغسطس 2024 ...,5.0 من 5 نجوم | 1.0 من 5 نجوم | 4.0 من 5 نجوم ...,Mostafa Ahmed | هيثم حسني سعد | user-MK5FTQ1 |...,وصل التليفون مظبوط واقل سعر معروض فى نسخه 8gb ...,3.5 من 5 نجوم
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3788,إلكترونيات,رقائق حماية الشاشة,شاشة حماية خلفية مرنة من ألياف كربونية مضادة ل...,NaN,NaN,45.‎.00,جينيريك,NaN,NaN,NaN,...,شاشة حماية خلفية لموبايل ابل ايفون,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D9%83%D8%B1%D8%A8%D9%88...,تمت كتابة هذه المراجعة في مصر في 10 أغسطس 2024...,1.0 من 5 نجوم | 5.0 من 5 نجوم | 3.0 من 5 نجوم ...,Yassin El Sayed | Pierre A. | Mohamed | BadR |...,اصغر من الموبايل\nاقرأ المزيد | It’s not total...,3.0 من 5 نجوم
3789,إلكترونيات,أغطية أساسية,جراب سائل بالوان قوس قزح لموبايل ايفون 15 من ا...,NaN,NaN,150.‎.00,جينيريك,NaN,NaN,NaN,...,حافظة سائل بألوان قوس قزح لهاتف ايفون 15 حافظة...,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D8%A8%D8%A7%D9%84%D9%88...,NaN,NaN,NaN,NaN,Previous page
3790,إلكترونيات,أغطية أساسية,غطاء حماية سميك مضاد للسقوط مطلي للهاتف المحمو...,NaN,NaN,259.‎.00,جينيريك,NaN,NaN,NaN,...,شفاف,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.e

In [127]:
df.head(30)

,category,subcategory,product_name,promo_details,price_before_promo,price_after_promo,phone_type,operating_system,storage,ram_memory,...,description,asin,unit_price,image_links,product_url,review_dates,review_ratings,reviewer_names,review_texts,whole_rating
0,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,موبايل سامسونج جالاكسي A15 ثنائي شرائح الاتصال...,-19‎%‎,‎10270.00‎,8299.‎.00,سامسونج,Android 13.0,256 غيغابايت,8 غيغابايت,...,الشاشة: تتميز شاشة سوبر اموليد مقاس 6.5 بوصة ل...,B0CQJZ76X9,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D9%85%D9%88%D8%A8%D8%A7...,تمت كتابة هذا التقييم في مصر في 19 أغسطس 2024 ...,5.0 من 5 نجوم | 5.0 من 5 نجوم | 1.0 من 5 نجوم ...,محمد حسين | إيناس ل. | Eng Ahmed Mesbah | افشل...,هاتف جيد وجاء كما هو مطلوب شكرا امازون\nاقرأ ا...,3.9 من 5 نجوم
1,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,جوال سامسونج جالكسي ايه 55 الجيل الخامس 5G، ثن...,NaN,NaN,19099.‎.00,سامسونج,Android 11.0,256 غيغابايت,8 غيغابايت,...,العلامة التجارية: سامسونج,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D8%B3%D8%A7%D9%85%D8%B3...,تمت كتابة هذا التقييم في مصر في 14 أغسطس 2024 ...,5.0 من 5 نجوم | 5.0 من 5 نجوم | 1.0 من 5 نجوم ...,Ahmed Baraka | Abanoub | أبانوب ميشيل | Buyer ...,موبايل إمكانياته محترمه ومثالي للاستخدام اليوم...,4.0 من 5 نجوم
2,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,هاتف ذكي ريدمي A3 بذاكرة RAM 3GB وذاكرة تخزين ...,-18‎%‎,‎4887.72‎,3999.‎.00,شاومي,Android 10.0,64 غيغابايت,NaN,...,يدعمي شريحتي اتصال وميكرو اس دي، يدعم نطاقات ش...,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D9%88%D8%A8%D8%B7%D8%A7...,تمت كتابة هذا التقييم في مصر في 24 يوليو 2024 ...,5.0 من 5 نجوم | 3.0 من 5 نجوم | 1.0 من 5 نجوم ...,Ahmed abdallah | Hadeer Haggag | ياسر عبدالحى ...,ممتاز جدا\nاقرأ المزيد | مناسب بالنسبه للسعر ل...,3.5 من 5 نجوم
3,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,موبايل سمارت سامسونج جالاكسي A55 الجيل الخامس ...,-23‎%‎,‎23499.00‎,17995.‎.00,سامسونج,Android 11.0,256 غيغابايت,8 غيغابايت,...,العلامة التجارية: سامسونج,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D8%B3%D8%A7%D9%85%D8%B3...,تمت كتابة هذا التقييم في مصر في 27 يونيو 2024,5.0 من 5 نجوم,أيمن خطاب,منتج جيد وبالتقسيط بدون فوائد ومغلق بشكل جيد\n...,4.1 من 5 نجوم
4,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,جوال شاومي ريدمي 13C ثنائي شرائح الاتصال بذاكر...,-13‎%‎,‎6777.00‎,5866.‎.24,شاومي,ام اي يو اي 14,128 غيغابايت,6 غيغابايت,...,شاشة عرض 6.74 بوصة 90 هرتز,B0CT3MBS1L,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D8%B4%D8%A7%D9%88%D9%85...,تمت كتابة هذا التقييم في مصر في 16 أغسطس 2024 ...,5.0 من 5 نجوم | 1.0 من 5 نجوم | 4.0 من 5 نجوم ...,Mostafa Ahmed | هيثم حسني سعد | user-MK5FTQ1 |...,وصل التليفون مظبوط واقل سعر معروض فى نسخه 8gb ...,3.5 من 5 نجوم
5,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,موبايل سامسونج جالاكسي A05s، يعمل بنظام اندروي...,NaN,NaN,5499.‎.00,سامسونج,Android 13.0,128 غيغابايت,4 غيغابايت,...,العلامة التجارية: سامسونج,NaN,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D9%85%D9%88%D8%A8%D8%A7...,تمت كتابة هذا التقييم في مصر في 20 أغسطس 2024 ...,5.0 من 5 نجوم | 5.0 من 5 نجوم | 5.0 من 5 نجوم ...,محمود الباشا | Mide | تمت الموافقه على المرتجع...,منتج ممتاز ويعمل بشكل جيد\nاقرأ المزيد | No co...,4.1 من 5 نجوم
6,إلكترونيات,الهواتف المحمولة خالية من بطاقة الاتصال وغير ا...,موبايل سامسونج جالاكسي A05s، هاتف ذكي يعمل بنظ...,NaN,NaN,5999.‎.00,سامسونج,Android 13.0,128 غيغابايت,6 غيغابايت,...,العلامة التجارية: سامسونج,B0CP469YWV,جنيه,https://images-na.ssl-images-amazon.com/images...,https://www.amazon.eg/%D8%B3%D8%A7%D9%85%D8%B3...,تمت كتابة هذا التقييم في مصر في 20 أغسطس 2024 ...,5.0 من 5 نجوم | 5.0 من 5 نجوم | 5.0 من 5 نجوم ...,محمود الباشا | Mide | تمت الموافقه على المرتجع...,منتج ممتاز ويعمل بشكل جيد\nاقرأ المزيد | No co...,4.1 من 5 نجوم
7,إلكترونيات,الهوات

In [130]:
#for excel sheet
df = pd.read_csv('products.csv')
df.to_excel('products.xlsx', index=False)

In [ ]:
#additional part

In [131]:
#for amazon.com
import csv
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

def get_url(search_term):
    template = 'https://www.amazon.com/s?k={}&ref=nb_sb_noss_2'
    search_term = search_term.replace(" ", "+")
    url = template.format(search_term)
    url += '&page={}'
    return url

def extract_record(item):
    atag = item.h2.a if item.h2 and item.h2.a else None
    description = atag.text.strip() if atag else "N/A"
    product_url = 'https://www.amazon.com' + atag.get('href') if atag else "N/A"

    try:
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        price = "N/A"
    
    try:
        rating = item.i.text if item.i else "N/A"
        review_count = item.find('span', {'class': 'a-size-base'}).text if item.find('span', {'class': 'a-size-base'}) else "N/A"
    except AttributeError:
        rating = "N/A"
        review_count = "N/A"

    result = (description, price, rating, review_count, product_url)
    return result

def main():
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service)

    brand = input('Enter the brand name:')
    product_type = input('Enter the product type:')
    search_term = f"{brand} {product_type}"
    records = []

    url = get_url(search_term)

    for page in range(1, 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})

        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)

    driver.quit()

    df = pd.DataFrame(records, columns=['Description', 'Price', 'Rating', 'Review Count', 'URL'])
    df.to_csv('productss.csv', index=False)

    print('Done')

if __name__ == "__main__":
    main()
df=pd.read_csv("productss.csv")
df.head(30)
df.dropna()
 

Enter the brand name:iphone
Enter the product type:mobile
Done


,Description,Price,Rating,Review Count,URL
22,iPhone 14 Pro Max Magnetic Case Compatible wit...,$21.00,3.5 out of 5 stars,20,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
39,"Metal Phone Tripod Mount w 2 Cold Shoes, Verti...",$22.99,4.1 out of 5 stars,52,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
42,"Dash Phone Holder for Car, Stable 360° Rotatab...",$19.97,5.0 out of 5 stars,2,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
60,"Selfie Light, 84 LED Phone Light, 5000mAh Rech...",$20.99,4.7 out of 5 stars,60,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
75,Grey Crystal iPhone11/12 Plus Case Diamond Uni...,$26.89,5.0 out of 5 stars,1,https://www.amazon.com/Crystal-iPhone-Unique-P...
80,"Phone Selfie Light with Clip, 142 LED Fill Lig...",$23.90,4.3 out of 5 stars,5,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
82,"USB C Car Charger Adapter, 12V Car Charger Mul...",$19.19,4.7 out of 5 stars,184,https://www.amazon.com/sspa/click?ie=UTF8&spc=...
84,Compatible with iPhone 14 Pro MAX Wallet Case ...,$64.99,5.0 out of 5 stars,1,https://www.amazon.com/Protected-Wireless-Comp...
86,"NEW iPhone 12, 12 Pro, 12 Pro Max, 12 Mini, sl...",$44.00,5.0 out of 5 stars,2,https://www.amazon.com/vintage-handmade-Crazy-...
87,"iPhone 12, 12 Pro, 12 Pro Max, 12 Mini, sleeve...",$44.00,5.0 out of 5 stars,2,https://www.amazon.com/vintage-handmade-Crazy-...
